In [11]:
import re

import numpy as np
import polars as pl

from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud

from parse_telegram_json_polars import parse_telegram_chat

In [12]:
STOPWORDS = ['',
 'только',
 'об',
 'если',
 'hasn',
 'на',
 'этому',
 'нередко',
 'само',
 'другой',
 'часто',
 'until',
 "she's",
 'their',
 "you're",
 'время',
 'вокруг',
 'оно',
 'седьмой',
 'или',
 'нх',
 'before',
 'две',
 'менее',
 'нас',
 'больше',
 'раз',
 'ему',
 'двадцатый',
 'an',
 'одной',
 'миллионов',
 'будь',
 'даже',
 'прекрасно',
 'этими',
 'сначала',
 'o',
 'между',
 'которая',
 'в',
 'восемнадцатый',
 'isn',
 'own',
 'занят',
 'нею',
 'значит',
 'после',
 'моя',
 'whom',
 'семнадцатый',
 'будет',
 'самой',
 'after',
 'пора',
 'тринадцатый',
 'or',
 'свое',
 'yours',
 'should',
 'у',
 'меля',
 'по',
 'под',
 'off',
 'сегодня',
 'but',
 'ваши',
 'внизу',
 'давно',
 'слишком',
 'бывь',
 'хотя',
 'from',
 'около',
 'хоть',
 'she',
 'by',
 'первый',
 'все',
 'ничего',
 'мои',
 'твоя',
 'семнадцать',
 'мира',
 'myself',
 'сказать',
 'год',
 'ваше',
 'above',
 'том',
 'other',
 'пятый',
 'over',
 'such',
 'лучше',
 'они',
 'which',
 'восемь',
 'многочисленный',
 'там',
 "shouldn't",
 'тебе',
 'тогда',
 'десятый',
 'нам',
 'та',
 'тобой',
 'сам',
 'it',
 "it's",
 'но',
 'too',
 'лишь',
 'being',
 'году',
 'шесть',
 'которой',
 'where',
 'более',
 'можно',
 'мог',
 'даром',
 'просто',
 'собой',
 'самими',
 'дел',
 'будто',
 'кто',
 'me',
 'more',
 'можхо',
 'during',
 'этом',
 'года',
 'in',
 'сами',
 "doesn't",
 'тою',
 'didn',
 'our',
 'своей',
 'было',
 'есть',
 'жизнь',
 'needn',
 'вам',
 'чтобы',
 'суть',
 'меньше',
 'наверху',
 'были',
 'семь',
 'будем',
 'can',
 'самом',
 'совсем',
 'зато',
 'down',
 'над',
 'уже',
 'когда',
 'процентов',
 'wouldn',
 'тобою',
 'вроде',
 'be',
 'aren',
 'этой',
 'theirs',
 'многочисленные',
 'нее',
 "won't",
 'ее',
 'пять',
 'they',
 'будете',
 'ли',
 'because',
 'алло',
 'хотеть',
 'них',
 'hadn',
 'иногда',
 'ей',
 'потом',
 'бывает',
 'сказал',
 'здесь',
 'both',
 'дальше',
 'then',
 'for',
 'мне',
 'that',
 'очень',
 'каждые',
 'лет',
 'день',
 'mightn',
 'друго',
 'as',
 'out',
 'http',
 'having',
 "needn't",
 'была',
 'ком',
 'четвертый',
 'мочь',
 'тысяч',
 't',
 'мы',
 'any',
 'вон',
 'ваша',
 'при',
 'тебя',
 'he',
 'чем',
 'very',
 'тем',
 'десять',
 'самого',
 "didn't",
 'who',
 'их',
 'him',
 'сеаой',
 'мной',
 'some',
 'о',
 'herself',
 'бы',
 'couldn',
 'on',
 'чтоб',
 'именно',
 'собою',
 'шестнадцать',
 "shan't",
 'такая',
 'недавно',
 'человек',
 'будут',
 'куда',
 'самих',
 'с',
 'мож',
 'ниже',
 'себя',
 'иметь',
 've',
 'shan',
 'ту',
 'сейчас',
 'другая',
 'few',
 'of',
 'will',
 "mustn't",
 'what',
 'https',
 'each',
 'одиннадцать',
 'из',
 'нет',
 'отовсюду',
 'опять',
 "don't",
 'нужно',
 'до',
 'вас',
 'хорошо',
 'занято',
 'девятнадцатый',
 'который',
 'может',
 'него',
 "haven't",
 'да',
 'when',
 'три',
 'несколько',
 'позже',
 'тех',
 'ну',
 'себе',
 'чего',
 'рядом',
 'снова',
 'нами',
 'много',
 "couldn't",
 'haven',
 'почему',
 'и',
 'девять',
 'his',
 'тут',
 'наша',
 'кого',
 'my',
 'его',
 'будешь',
 'we',
 'wasn',
 'однако',
 'всю',
 'about',
 'if',
 'ours',
 'одиннадцатый',
 'does',
 'ain',
 'саму',
 'doesn',
 'ведь',
 'here',
 'mustn',
 'did',
 'пока',
 'ourselves',
 'the',
 'долго',
 'теми',
 'тринадцать',
 'важное',
 'нибудь',
 'важная',
 'того',
 'конечно',
 'won',
 'туда',
 'вот',
 'himself',
 'your',
 'вдали',
 'ни',
 'еще',
 'сих',
 'от',
 'i',
 'наш',
 'how',
 'низко',
 'y',
 'were',
 'назад',
 'so',
 'between',
 'am',
 'кругом',
 'эти',
 'are',
 'другие',
 'чуть',
 "should've",
 'заняты',
 'уж',
 'посреди',
 'этого',
 'same',
 'такой',
 're',
 'девятнадцать',
 'которые',
 'так',
 'ней',
 'твой',
 'мор',
 'вся',
 'люди',
 'довольно',
 'com',
 'затем',
 'themselves',
 'ваш',
 'yourself',
 'those',
 'со',
 'into',
 'not',
 'четырнадцать',
 "you'll",
 'was',
 'all',
 'had',
 'оба',
 'кому',
 'начала',
 "isn't",
 'рано',
 'моё',
 'своего',
 'стал',
 'должно',
 'потому',
 "you've",
 'm',
 'я',
 'тоже',
 'каждая',
 "hasn't",
 'only',
 'ними',
 'восемнадцать',
 'зачем',
 'with',
 'всего',
 "aren't",
 'её',
 'про',
 'против',
 'к',
 'нельзя',
 'just',
 'while',
 'надо',
 'пор',
 'шестнадцатый',
 'вверх',
 'below',
 'yourselves',
 'have',
 'наконец',
 'важный',
 'имя',
 'мною',
 'могут',
 'now',
 'ею',
 'далеко',
 'других',
 'всею',
 'за',
 'этих',
 'кажется',
 "wouldn't",
 'and',
 'самим',
 'хочешь',
 'do',
 'her',
 'всех',
 'тот',
 'буду',
 'непрерывно',
 'itself',
 'какой',
 'впрочем',
 'ним',
 'пятнадцать',
 'один',
 'ещё',
 'четырнадцатый',
 'весь',
 'всему',
 'не',
 'занята',
 'weren',
 'its',
 'наши',
 'мой',
 'you',
 'почти',
 'has',
 'вы',
 'раньше',
 'которого',
 'этим',
 'hers',
 "you'd",
 'most',
 'двенадцатый',
 'говорил',
 'сказала',
 'нем',
 'что',
 'такое',
 'это',
 'вами',
 'теперь',
 'многочисленное',
 'once',
 'under',
 'те',
 'shouldn',
 'разве',
 'свои',
 "weren't",
 'везде',
 'don',
 'для',
 'чаще',
 'близко',
 'неё',
 'кроме',
 'два',
 'действительно',
 'всем',
 'многочисленная',
 'наше',
 'against',
 'чему',
 'им',
 'пятнадцатый',
 'говорит',
 'тому',
 'вообще',
 'двенадцать',
 'further',
 'всюду',
 'd',
 'самому',
 'недалеко',
 'кем',
 'no',
 "mightn't",
 'такие',
 'them',
 'обычно',
 'немного',
 'девятый',
 'мало',
 'одного',
 'времени',
 'где',
 'то',
 'again',
 'он',
 'up',
 'второй',
 'каждый',
 'вдруг',
 'наиболее',
 'через',
 'пожалуйста',
 'также',
 'is',
 'ж',
 'каждое',
 'шестой',
 'другое',
 'свою',
 'был',
 'уметь',
 "hadn't",
 'которых',
 'она',
 'нему',
 'как',
 'восьмой',
 'ma',
 'than',
 'своих',
 'без',
 'сколько',
 'отсюда',
 'through',
 'твоё',
 'а',
 'во',
 'важные',
 'these',
 'двадцать',
 "that'll",
 'третий',
 'этот',
 'никогда',
 'вниз',
 'мимо',
 'быть',
 'меня',
 'четыре',
 'всеми',
 'особенно',
 'to',
 'всё',
 'this',
 's',
 'ты',
 'сама',
 'эту',
 'спасибо',
 'никуда',
 'why',
 'a',
 'эта',
 'же',
 'there',
 'какая',
 'перед',
 "wasn't",
 'ими',
 'doing',
 'двух',
 'всегда',
 'at',
 'однажды',
 'been',
 'nor',
 'll']

In [14]:
FILE_PATH = '/home/dreamtim/Coding/saved-ai-bot/exported_chats/result_germany.json'

In [13]:
def clean_text(text, for_rf_idf=False):

    text = text.replace('(File not included. Change data exporting settings to download.)', '')

    # Clear characters other than numbers and letters of the alphabet only (english and russian)
    text = re.sub(r'[^A-Za-zА-Яа-яЁё ]', '', text, flags=re.IGNORECASE)

    if for_rf_idf:
        return ' '.join(sorted(text.split()))
    
    return text

In [17]:
df, chat_name = parse_telegram_chat(FILE_PATH)

In [ ]:
def generate_wordcloud(df, chat_name):

    vectorizer = TfidfVectorizer(stop_words=STOPWORDS)
    tfidf_matrix = vectorizer.fit_transform(df['msg_content'].map_elements(clean_text, return_dtype=pl.Object))

    column_sums = np.array(tfidf_matrix.sum(axis=0)).flatten()
    column_names = vectorizer.get_feature_names_out()
    word_frequencies = dict(zip(column_names, column_sums))

    wordcloud = WordCloud(max_font_size=80, max_words=100, stopwords=STOPWORDS, min_font_size=15)
    wordcloud.generate_from_frequencies(word_frequencies)
    wordcloud = WordCloud(max_font_size=80, max_words=100, stopwords=STOPWORDS, min_font_size=15)
    wordcloud.to_file(f'wordclouds/{chat_name}_wordcloud.png')

In [33]:
wordcloud.to_file(f'wordclouds/{chat_name}_wordcloud.png')